In [ ]:
!pip install google_trans_new

In [1]:
import findspark
findspark.init()
import pyspark
import traceback
import json
from pyspark.sql.session import SparkSession
import os
from google_trans_new import google_translator  
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

conf = pyspark.SparkConf()
conf.set("spark.executor.extraClassPath",  os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))
conf.set("spark.driver.extraClassPath", os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))

sc = pyspark.SparkContext(appName="TwitterStreamApp", conf=conf)

spark = SparkSession(sc)
translator = google_translator()  

In [2]:
url = 'jdbc:sqlite:' + os.path.join(os.getcwd(), 'database.sqlite')

df = spark.read.jdbc(url, 'Tweet')
df.createOrReplaceTempView('Tweet')

In [3]:
!export JAVA_HOME='/usr/lib/jvm/java-8-openjdk-amd64'
!export PATH=$JAVA_HOME/bin:$PATH

In [4]:
def sentiment_score(tweet):
  
    sid = SentimentIntensityAnalyzer()
  
    sentiment = sid.polarity_scores(tweet)
      
    if sentiment['compound'] >= 0.05:
        return "Positive"
    elif sentiment['compound'] <= - 0.05:
        return "Negative"
    else:
        return 'Neutral'

In [5]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        sql_context = get_sql_context_instance(rdd.context)

        row_rdd = rdd.map(lambda x: (x, translator.translate(
                x['extended_tweet']['full_text'] if x['truncated'] else x['text'], lang_src='es', lang_tgt='en'
            ))).map(lambda x: Row(
            name=x[0]['user']['screen_name'],
            date=datetime.datetime.fromtimestamp(int(x[0]['timestamp_ms']) / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            text=x[0]['extended_tweet']['full_text'] if x[0]['truncated'] else x[0]['text'],
            transtext=x[1],
            sentiment=sentiment_score(x[1])
        ))

        tweets_df = sql_context.createDataFrame(row_rdd)
        tweets_df.write.mode("append").jdbc(url, 'Tweet')
    except:
        pass

In [6]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests

In [19]:

sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint_TwitterApp")
dataStream = ssc.socketTextStream("localhost",9005)
tweets = dataStream.map(lambda x: json.loads(x))

tweets.foreachRDD(process_rdd)
mySsc = ssc

ssc.start()
ssc.awaitTermination()


----------- 2021-05-16 16:17:24 -----------
----------- 2021-05-16 16:17:26 -----------
----------- 2021-05-16 16:17:28 -----------
----------- 2021-05-16 16:17:30 -----------
----------- 2021-05-16 16:17:32 -----------
----------- 2021-05-16 16:17:34 -----------
----------- 2021-05-16 16:17:36 -----------
----------- 2021-05-16 16:17:38 -----------
----------- 2021-05-16 16:17:40 -----------
----------- 2021-05-16 16:17:42 -----------
----------- 2021-05-16 16:17:44 -----------
----------- 2021-05-16 16:17:46 -----------
----------- 2021-05-16 16:17:48 -----------
----------- 2021-05-16 16:17:50 -----------
----------- 2021-05-16 16:17:52 -----------
----------- 2021-05-16 16:17:54 -----------
----------- 2021-05-16 16:17:56 -----------
----------- 2021-05-16 16:17:58 -----------
----------- 2021-05-16 16:18:00 -----------
----------- 2021-05-16 16:18:02 -----------
----------- 2021-05-16 16:18:04 -----------


KeyboardInterrupt: 

In [21]:
ssc.stop(False)